In [1]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense, Convolution2D, ELU, Dropout
from keras.models import Model, Sequential
from keras import backend
from keras import callbacks
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import cv2
import matplotlib.image as mpimg
from random import shuffle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import json
%matplotlib inline

Using TensorFlow backend.


In [2]:
tf.python.control_flow_ops = tf
backend.image_dim_ordering()

'tf'

In [3]:
def save_model(model, model_name):
    model.save_weights(model_name+".h5", True)
    with open(model_name+'.json', 'w') as outfile:
        json.dump(model.to_json(), outfile)

In [4]:
def make_model_vgg16():
    # vgg16
    model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

    #Create input format
    input = Input(name = 'image_input', shape=(224,224,3))

    #Use the generated model 
    output_vgg16_conv = model_vgg16_conv(input)

    # freeze vgg16 conv layers
    for layer in model_vgg16_conv.layers:
        layer.trainable = False

    #model_vgg16_conv.summary()

    # Add the fully-connected layers 
    x = Flatten(name='flatten')(output_vgg16_conv)
    x = Dense(256, activation='relu', name='fc1')(x)
    x = Dense(256, activation='relu', name='fc2')(x)
    x = Dense(1, activation='linear', name='predicton_steering')(x)

    # Create model 
    return Model(input=input, output=x)

In [5]:
def make_model_nvidia():
    # based on this paper:
    # http://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf
    input = Input(shape=(80,160,3),name = 'image_input')

    x=Convolution2D(24, 5, 5, border_mode="valid", subsample=(2, 2), activation="relu", name='conv1')(input)
    x=Convolution2D(36, 5, 5, border_mode="valid", subsample=(2, 2), activation="relu", name='conv2')(x)
    x=Convolution2D(48, 5, 5, border_mode="valid", subsample=(2, 2), activation="relu", name='conv3')(x)
    x=Convolution2D(64, 3, 3, border_mode="valid", subsample=(1, 1), activation="relu", name='conv4')(x)
    x=Convolution2D(64, 3, 3, border_mode="valid", subsample=(1, 1), activation="relu", name='conv5')(x)
    x=Flatten(name='flatten')(x) 
    x=Dense(1164, activation="relu", name='dense1')(x)
    x=Dense(100, activation="relu", name='dense2')(x)
    x=Dense(50, activation="relu", name='dense3')(x)
    x=Dense(10, activation="relu", name='dense4')(x)
    x=Dense(1, name='dense5')(x)
    
    # Create model
    return Model(input=input, output=x)

In [6]:
def make_model_commaai():
    model = Sequential()
    #model.add(Lambda(lambda x: x/127.5 - 1.,
    #        input_shape=(ch, row, col),
    #        output_shape=(ch, row, col)))
    model.add(Convolution2D(16, 8, 8, subsample=(4, 4), border_mode="same", input_shape = (80,160,3)))
    model.add(ELU())
    model.add(Convolution2D(32, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(ELU())
    model.add(Convolution2D(64, 5, 5, subsample=(2, 2), border_mode="same"))
    model.add(Flatten())
    model.add(Dropout(.2))
    model.add(ELU())
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(1))

    #model.compile(optimizer="adam", loss="mse")
    return model


In [7]:
my_model=make_model_nvidia()
my_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
image_input (InputLayer)         (None, 80, 160, 3)    0                                            
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 38, 78, 24)    1824        image_input[0][0]                
____________________________________________________________________________________________________
conv2 (Convolution2D)            (None, 17, 37, 36)    21636       conv1[0][0]                      
____________________________________________________________________________________________________
conv3 (Convolution2D)            (None, 7, 17, 48)     43248       conv2[0][0]                      
___________________________________________________________________________________________

In [8]:
base_path = os.getcwd()

In [ ]:
log = pd.read_csv(base_path + '/data/udacity/data/' + 'driving_log.csv')
log = log.drop(log[abs(log['steering']) < 0.1].index)
#log['steering']

In [9]:
def preprocess_data_log(base_path, csv_file_name, split=0.1, col_name='center', steer_mod=0.0, flip_th=None):
    # read log file
    log = pd.read_csv(base_path + '/' + csv_file_name)
    
    # drop rows base don flip_th
    if flip_th is None:
        flip = [0]*len(log)
    else:
        log = log.drop(log[abs(log['steering']) < flip_th].index)
        flip = [1]*len(log)   
    
    # modify img file name column to contain the full path
    log[col_name] = base_path + '/' + log[col_name].str.lstrip()
    # extract img file names
    img_names = log[col_name].tolist()
    # extract angles
    angles = log['steering']   
    angles += steer_mod
    
    img_names_train, img_names_test, angles_train, angles_test = train_test_split(img_names, angles, test_size=split, random_state=42)
    
    return ( list(zip(img_names_train, angles_train, flip)), list(zip(img_names_test, angles_test, flip)) )

In [10]:
# center images
train_data_list, test_data_list = preprocess_data_log(base_path + '/data/udacity/data', 'driving_log.csv')

# left images
train_data_list_l, test_data_list_l = preprocess_data_log(base_path + '/data/udacity/data', 'driving_log.csv',
                                                          col_name='left', steer_mod=0.15)
# right images
train_data_list_r, test_data_list_r = preprocess_data_log(base_path + '/data/udacity/data', 'driving_log.csv',
                                                          col_name='right', steer_mod=-0.15)

# center images to be flipped
train_data_list_f, test_data_list_f = preprocess_data_log(base_path + '/data/udacity/data', 'driving_log.csv',
                                                      flip_th=0.1)

# left images to be flipped
train_data_list_l_f, test_data_list_l_f = preprocess_data_log(base_path + '/data/udacity/data', 'driving_log.csv',
                                                          col_name='left', steer_mod=0.15, flip_th=0.1)

# right images to be flipped
train_data_list_r_f, test_data_list_r_f = preprocess_data_log(base_path + '/data/udacity/data', 'driving_log.csv',
                                                          col_name='right', steer_mod=-0.15, flip_th=0.1)

In [11]:
train_data_list = train_data_list +\
                    train_data_list_l +\
                    train_data_list_r +\
                    train_data_list_f +\
                    train_data_list_l_f +\
                    train_data_list_r_f


test_data_list = test_data_list +\
                    test_data_list_l +\
                    test_data_list_r +\
                    test_data_list_f +\
                    test_data_list_l_f +\
                    test_data_list_r_f

In [12]:
train_data_list_f[200]

('/home/carnd/CarND-Behavioral-Cloning-Project/data/udacity/data/IMG/center_2016_12_01_13_41_47_802.jpg',
 -0.26863309999999996,
 1)

In [13]:
my_model.compile(optimizer='adam', loss='mse')
#my_model.compile(optimizer='adam', loss='mean_absolute_percentage_error', metrics=['accuracy'])
#my_model.compile(optimizer='adam', loss='mse')

In [14]:
def batch_gen(name, data_list, out_img_size, batch_size):
    # data_list is in this form: ['file_name', angle]
    # out_img_size is a shape of the output image: (x, y, color)
    
    # create batch_size np arrays as placeholders for imgages and angles
    X_train = np.empty((batch_size,) +  out_img_size, dtype = np.float32)
    y_train = np.empty(batch_size, dtype = np.float32)
    
    # col & rows are reversed for cv2.resize
    resize_shape = (out_img_size[1], out_img_size[0])
    
    while True:
        # shuffle data
        shuffle(data_list)
        for offset in range(0, len(data_list), batch_size):
            end = offset + batch_size
            batch_subset = data_list[offset:end]
            for i, _ in enumerate(batch_subset):
                # load angle
                y_train[i] = batch_subset[i][1]
                
                #load image
                im = cv2.imread(batch_subset[i][0])
                
                # flip it if required
                if batch_subset[i][2] == 1:
                    im = cv2.flip(im, 1)
                    y_train[i] *= -1.
                    
                # resize and norm
                X_train[i] = (cv2.resize(im, resize_shape).astype(np.float32))/128. - 1.
                #X_train[i] = np.expand_dims(im, axis=0)
                
                
            batch_X, batch_y = X_train[:len(batch_subset)], y_train[:len(batch_subset)]
            #print(name)
            yield (batch_X, batch_y)

In [15]:
#g=batch_gen(out[0:6], (160, 320, 3), 2)

In [16]:
#(im, a) = next(g)
len(train_data_list)

27399

In [17]:
len(test_data_list)

3048

In [18]:
#plt.imshow(cv2.cvtColor((im[0]*255.+128.).astype(np.uint8), cv2.COLOR_BGR2RGB))
#im0 = cv2.resize(cv2.imread(train_data_list[0][0]), (224, 224))
#plt.imshow(cv2.cvtColor(im0, cv2.COLOR_BGR2RGB))

In [19]:
train_gen = batch_gen("train_gen", train_data_list, (80, 160, 3), 128)
test_gen = batch_gen("test_gen", test_data_list, (80, 160, 3), 128)

In [24]:
history = my_model.fit_generator(
    generator=train_gen, 
    validation_data=test_gen, 
    nb_val_samples=len(test_data_list), 
    samples_per_epoch=len(train_data_list), 
    nb_epoch=7,
    max_q_size=1)

Epoch 1/7
 3072/27399 [==>...........................] - ETA: 37s - loss: 0.0059

KeyboardInterrupt: 

In [23]:
save_model(my_model, 'nvidia')

In [ ]:
train_gen = batch_gen("train_gen", train_data_list, (80, 160, 3), 1)

In [ ]:
im, t = next(train_gen)

In [ ]:
#im[0]

In [ ]:
im0 = (im[0]*128.+128.).astype(np.uint8)

In [ ]:
train_gen = batch_gen("train_gen", train_data_list, (80, 160, 3), 25)

In [ ]:
im, s = next(train_gen)

In [ ]:
my_model.predict(im)

In [ ]:
s